In [23]:
from transformers import MarianMTModel, MarianTokenizer

In [47]:
model_name = "Helsinki-NLP/opus-mt-en-roa"
model_name = "Helsinki-NLP/opus-mt-en-es"
model_name = "Helsinki-NLP/opus-mt-tc-big-en-es"
model_name = "Helsinki-NLP/opus-mt-es-en"


In [57]:
src_text = [
    "A wasp stung him and he had an allergic reaction.",
    "I love nature."
]
src_text = [
    "Amo la naturaleza.",
    "soy Steven de Ecuador y me gusta el futbol",
    "Una avispa lo picó y tuvo una reacción alérgica."
]

In [58]:
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [59]:
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

In [60]:
for t in translated:
    print( tokenizer.decode(t, skip_special_tokens=True) )

# expected output:
#     Una avispa lo picó y tuvo una reacción alérgica.
#     Me encanta la naturaleza.

I love nature.
I'm Steven from Ecuador and I like football
A wasp stung him and had an allergic reaction.


In [56]:
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['I love nature.', "I'm Steven from Ecuador and I like football"]

In [ ]:
# from huggingface_hub import list_models

# model_list = list_models()

In [ ]:
# org = "Helsinki-NLP"
# model_ids = [x.id for x in model_list if x.id.startswith(org)]
# suffix = [x.split("/")[1] for x in model_ids]
# old_style_multi_models = [f"{org}/{s}" for s in suffix if s != s.lower()]

In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-es")

frluquba

In [61]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="frluquba/traductor_es_en")

Device set to use cpu


In [62]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("frluquba/traductor_es_en")
model = AutoModelForSeq2SeqLM.from_pretrained("frluquba/traductor_es_en")

In [ ]:
src_text = [
    "Amo la naturaleza.",
    "soy Steven de Ecuador y me gusta el futbol",
    "Una avispa lo picó y tuvo una reacción alérgica."
]

In [63]:
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

In [64]:
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['Feelings of nature ',
 "i'm Steven of Ecuador and i'm passionate about football. ",
 'The rabbit was peeled and had a mild reaction. ']

['Feelings of nature ',  
 "i'm Steven of Ecuador and i'm passionate about football. ",  
 'The rabbit was peeled and had a mild reaction. ']

VS  

 I love nature.  
I'm Steven from Ecuador and I like football  
A wasp stung him and had an allergic reaction.  

In [ ]:
# from transformers import MarianMTModel, MarianTokenizer

# src_text = [
#     "A wasp stung him and he had an allergic reaction.",
#     "I love nature."
# ]

# model_name = "pytorch-models/opus-mt-tc-big-en-es"
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

# for t in translated:
#     print( tokenizer.decode(t, skip_special_tokens=True) )

# # expected output:
# #     Una avispa lo picó y tuvo una reacción alérgica.
# #     Me encanta la naturaleza.


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
import re

In [ ]:
TRADUCTOR_PATH = './traductor/'

In [ ]:
traductor_tokenizer = MarianTokenizer.from_pretrained(TRADUCTOR_PATH)
traductor_modelo    = MarianMTModel.from_pretrained(TRADUCTOR_PATH)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
def dividir_texto(texto, max_caracteres=1000):
    oraciones = re.split(r'(?<=[.!?]) +', texto)

    fragmentos = []
    fragmento_actual = ""

    for oracion in oraciones:
        if len(fragmento_actual) + len(oracion) <= max_caracteres:
            fragmento_actual += oracion + " "
        else:
            fragmentos.append(fragmento_actual.strip())
            fragmento_actual = oracion + " "

    if fragmento_actual:
        fragmentos.append(fragmento_actual.strip())

    return fragmentos

In [ ]:
def traducir_fragmentos(fragmentos):
    traducciones = []

    for fragmento in fragmentos:
        texto_codificado             = traductor_tokenizer(fragmento, return_tensors="pt")
        texto_codificado_traducido   = traductor_modelo.generate(**texto_codificado)
        texto_traducido_decodificado = traductor_tokenizer.batch_decode(texto_codificado_traducido, skip_special_tokens=True)
        traduccion_str               = texto_traducido_decodificado[0]
        traducciones.append(traduccion_str)

    return " ".join(traducciones)

In [ ]:
def traducir_texto(texto, max_caracteres=1000):
    fragmentos          = dividir_texto(texto, max_caracteres=max_caracteres)
    traduccion_completa = traducir_fragmentos(fragmentos)

    return traduccion_completa